In [14]:
!pip install python-dotenv
!pip install langchain
!pip install openai
!pip install langchain_openai
!pip install langchain_community

In [15]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
from langchain.tools.render import render_text_description_and_args
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit

In [16]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [18]:
!pip install amadeus

  Preparing metadata (setup.py) ... done
  Created wheel for amadeus: filename=amadeus-10.1.0-py2.py3-none-any.whl size=73273 sha256=9df2565be8d50cdafc2c7b45847d572248e06578ae72c14d1e4d5d86f4d32da2
  Stored in directory: /root/.cache/pip/wheels/57/a8/cf/279191bf9b7d8ca732be2b9a76ccf3b0e73768292f4fc0af4b
Successfully built amadeus


In [24]:
os.environ['OPENAI_API_KEY'] = '....'
os.environ['AMADEUS_CLIENT_ID']='...'
os.environ['AMADEUS_CLIENT_SECRET']='...'

In [25]:
from langchain_community.agent_toolkits.amadeus.toolkit import AmadeusToolkit
toolkit = AmadeusToolkit()
tools = toolkit.get_tools()

In [26]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name='gpt-4')

In [27]:
from langchain import hub
prompt = hub.pull("hwchase17/react-json")

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [28]:
from langchain.agents import create_react_agent
from langchain.agents.output_parsers import ReActJsonSingleInputOutputParser
agent = create_react_agent(
    llm,
    tools,
    prompt,
    tools_renderer=render_text_description_and_args,
    output_parser=ReActJsonSingleInputOutputParser(),
)

In [29]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [30]:
agent_executor.invoke({"input": "I need to travel from dubai to london on 1st april 2024. What are the cheapest options available"})



> Entering new AgentExecutor chain...
Thought: To find the cheapest flight options, I first need to find the airport codes for Dubai and London. I'll start with Dubai.
Action:
```
{
  "action": "closest_airport",
  "action_input": {
    "location": "Dubai"
  }
}
```content='{\n    "iataCode": "DXB"\n}' response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 54, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a951c4d4-df67-4c15-a112-7f2f30080f67-0'The airport code for Dubai is DXB. Now I need to find the airport code for London.
Action:
```
{
  "action": "closest_airport",
  "action_input": {
    "location": "London"
  }
}
```content='{\n  "iataCode": "LHR"\n}' response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 54, 'total_tokens': 66}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-7217ae11-b1a6

{'input': 'I need to travel from dubai to london on 1st april 2024. What are the cheapest options available',
 'output': "I'm sorry, but I can't provide the flight details at the moment. However, you can use the airport codes DXB for Dubai and LHR for London to search for flights on your preferred travel booking platform. Make sure to set your departure date to 1st April 2024 to find the cheapest options available."}

In [ ]:
st.title("Fly Smarter: Get Answers to All Your Flight Questions")
question_input = st.text_area("Enter your flight-related inquiry below.")
if st.button("Submit"):

    answer=agent_executor.invoke({"input": question_input})
    st.markdown(answer['output'])